In [3]:
from sfepy.discrete import Problem
from sfepy.discrete.fem import Mesh, FEDomain
from sfepy.discrete import (FieldVariable, Material, Integral, Function, Equation, Equations)
from sfepy.terms import Term
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton

In [4]:
# Load the .med mesh directly
mesh = Mesh.from_file('AMAZE.med')
domain = FEDomain('domain', mesh)

sfepy: reading mesh (AMAZE.med)...
sfepy:   number of vertices: 29133
sfepy:   number of cells:
sfepy:     1_2: 1753
sfepy:     3_4: 131622
sfepy:     2_3: 25990
sfepy: ...done in 0.05 s
sfepy: warning: mesh with topological dimension 1 lower than space dimension 3
sfepy: - element orientation not checked!


In [7]:
# Define regions using the named groups from the .med file
regions = {
    'Omega': 'all',
    'Gamma_PipeFace': ('facet by name PipeFace', 'facet'),  # Use the group name directly
    'Gamma_TileTop': ('facet by name TileTop', 'facet'),  # Use the group name directly
}

materials = {
    'mat': ({'k': 1.0},),
}

fields = {
    'temperature': ('real', 1, 'Omega', 1),
}

variables = {
    'T': ('unknown field', 'temperature', 0),
    'v': ('test field', 'temperature', 'T'),
}

ebcs = {
    'TileTop': ('Gamma_TileTop', {'T.0': 1.0}),
    'PipeFace': ('Gamma_PipeFace', {'T.0': -1.0}),
}

integrals = {
    'i': 2,
}

solvers = {
    'ls': ('ls.scipy_direct', {}),
    'newton': ('nls.newton', {
        'i_max': 1,
        'eps_a': 1e-10,
    }),
}


In [8]:
# Initialize the problem without passing configurations in the constructor
problem = Problem('heat_conduction_problem')

# Prepare equations as a dictionary
equations_dict = {
    'heat_conduction': 'dw_laplace.i.Omega(mat.k, v, T) = 0',
}

# Now, set up the problem with all components
problem.setup(domain=domain,
              fields=fields,
              materials=materials,
              equations=Equations.from_conf(equations_dict, problem.fields),
              integrals=integrals,
              solvers=solvers,
              bcs=Conditions.from_conf(ebcs, problem.fields))

# Optionally, for manual control, you can initialize solvers and equations like this:
problem.init_solvers()

# Solve the problem
problem.solve()

ValueError: missing equations in auto_conf mode!